# Air Quality PM2.5 Prediction - Model Training & Evaluation


## Objectives
1. Load processed training and test data
2. Train baseline models (Linear Regression, Ridge, Lasso)
3. Train tree-based models (Random Forest, Gradient Boosting, XGBoost)
4. Train advanced models (LightGBM, CatBoost)
5. Perform hyperparameter tuning
6. Compare model performance
7. Analyze residuals and predictions
8. Select and save best model

## Evaluation Metrics
- **RMSE** (Root Mean Squared Error) - Lower is better
- **MAE** (Mean Absolute Error) - Lower is better
- **R²** (R-squared) - Higher is better (max 1.0)
- **MAPE** (Mean Absolute Percentage Error) - Lower is better

## Models to Train
1. Linear Regression (Baseline)
2. Ridge Regression (L2 regularization)
3. Lasso Regression (L1 regularization)
4. ElasticNet (L1 + L2 regularization)
5. Random Forest Regressor
6. Gradient Boosting Regressor
7. XGBoost Regressor
8. LightGBM Regressor
9. CatBoost Regressor

---

In [ ]:
# ============================================
# IMPORT LIBRARIES
# ============================================

# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Sklearn - Preprocessing & Metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score,
    mean_absolute_percentage_error
)

# Sklearn - Model Selection
from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    GridSearchCV,
    RandomizedSearchCV,
    KFold
)

# Sklearn - Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Utilities
import pickle
import json
import os
from scipy import stats

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10



# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# ============================================
# LOAD PROCESSED DATA
# ============================================

print("="*60)
print("LOADING PROCESSED DATA")
print("="*60)

# Define paths
DATA_PATH = '../data/processed/'

print(f"\n Loading data from: {DATA_PATH}")

# Load training data
X_train = pd.read_csv(f'{DATA_PATH}X_train.csv')
y_train = pd.read_csv(f'{DATA_PATH}y_train.csv').values.ravel()  # Flatten to 1D array
y_train_original = pd.read_csv(f'{DATA_PATH}y_train_original.csv').values.ravel()

# Load test data
X_test = pd.read_csv(f'{DATA_PATH}X_test.csv')
y_test = pd.read_csv(f'{DATA_PATH}y_test.csv').values.ravel()
y_test_original = pd.read_csv(f'{DATA_PATH}y_test_original.csv').values.ravel()

# Load metadata
metadata_train = pd.read_csv(f'{DATA_PATH}metadata_train.csv')
metadata_test = pd.read_csv(f'{DATA_PATH}metadata_test.csv')

print(f"\n Data loaded successfully!")

print(f"\n Dataset Shapes:")
print(f"{'='*60}")
print(f"Training Features (X_train): {X_train.shape}")
print(f"Training Target (y_train): {y_train.shape}")
print(f"Test Features (X_test): {X_test.shape}")
print(f"Test Target (y_test): {y_test.shape}")

print(f"\n Target Statistics:")
print(f"{'='*60}")
print(f"Training Target (log-transformed):")
print(f"   Mean: {y_train.mean():.3f}")
print(f"   Std: {y_train.std():.3f}")
print(f"   Min: {y_train.min():.3f}")
print(f"   Max: {y_train.max():.3f}")

print(f"\nTraining Target (original scale):")
print(f"   Mean: {y_train_original.mean():.2f} μg/m³")
print(f"   Std: {y_train_original.std():.2f} μg/m³")
print(f"   Min: {y_train_original.min():.2f} μg/m³")
print(f"   Max: {y_train_original.max():.2f} μg/m³")

print(f"\n Feature Information:")
print(f"{'='*60}")
print(f"Number of features: {X_train.shape[1]}")
print(f"\nFirst 10 features:")
for i, feature in enumerate(X_train.columns[:10], 1):
    print(f"   {i}. {feature}")
if len(X_train.columns) > 10:
    print(f"   ... and {len(X_train.columns) - 10} more features")

# Check for missing values
missing_train = X_train.isnull().sum().sum()
missing_test = X_test.isnull().sum().sum()

if missing_train > 0 or missing_test > 0:
    print(f"\n Warning: Missing values detected!")
    print(f"   Training: {missing_train}")
    print(f"   Test: {missing_test}")
else:
    print(f"\n No missing values!")

# Display sample
print(f"\n Sample Data (First 5 rows):")
display(X_train.head())

In [ ]:
# ============================================
# FEATURE SCALING
# ============================================

print("="*60)
print("FEATURE SCALING")
print("="*60)

print(f"\n Standardizing features using StandardScaler...")
print(f"   (Mean = 0, Std = 1)")

# Initialize scaler
scaler = StandardScaler()

# Fit on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"\n Feature scaling completed!")

print(f"\n Scaled Features Statistics:")
print(f"   Training mean: {X_train_scaled.mean().mean():.6f}")
print(f"   Training std: {X_train_scaled.std().mean():.6f}")
print(f"   Test mean: {X_test_scaled.mean().mean():.6f}")
print(f"   Test std: {X_test_scaled.std().mean():.6f}")

# Visualize scaling effect
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Before scaling - first feature
feature_name = X_train.columns[0]
axes[0].hist(X_train[feature_name], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_xlabel(feature_name[:40])
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Before Scaling: {feature_name[:30]}...', fontweight='bold')
axes[0].grid(alpha=0.3)

# After scaling - same feature
axes[1].hist(X_train_scaled[feature_name], bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1].set_xlabel(f'{feature_name[:40]} (scaled)')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'After Scaling: {feature_name[:30]}...', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# ============================================
# EVALUATION FUNCTIONS
# ============================================

print("="*60)
print("DEFINING EVALUATION FUNCTIONS")
print("="*60)

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Evaluate a trained model and return metrics
    """
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # MAPE (avoid division by zero)
    train_mape = np.mean(np.abs((y_train - y_train_pred) / (y_train + 1e-10))) * 100
    test_mape = np.mean(np.abs((y_test - y_test_pred) / (y_test + 1e-10))) * 100
    
    # Store results
    results = {
        'Model': model_name,
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Train_MAE': train_mae,
        'Test_MAE': test_mae,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Train_MAPE': train_mape,
        'Test_MAPE': test_mape,
        'Predictions_Train': y_train_pred,
        'Predictions_Test': y_test_pred
    }
    
    return results

def print_evaluation(results):
    """
    Print evaluation metrics in a formatted way
    """
    print(f"\n{'='*60}")
    print(f"MODEL: {results['Model']}")
    print(f"{'='*60}")
    
    print(f"\n RMSE (Root Mean Squared Error):")
    print(f"   Training:   {results['Train_RMSE']:.4f}")
    print(f"   Test:       {results['Test_RMSE']:.4f}")
    print(f"   Difference: {abs(results['Train_RMSE'] - results['Test_RMSE']):.4f}")
    
    print(f"\n MAE (Mean Absolute Error):")
    print(f"   Training:   {results['Train_MAE']:.4f}")
    print(f"   Test:       {results['Test_MAE']:.4f}")
    print(f"   Difference: {abs(results['Train_MAE'] - results['Test_MAE']):.4f}")
    
    print(f"\n R² (R-squared):")
    print(f"   Training:   {results['Train_R2']:.4f}")
    print(f"   Test:       {results['Test_R2']:.4f}")
    print(f"   Difference: {abs(results['Train_R2'] - results['Test_R2']):.4f}")
    
    print(f"\n MAPE (Mean Absolute Percentage Error):")
    print(f"   Training:   {results['Train_MAPE']:.2f}%")
    print(f"   Test:       {results['Test_MAPE']:.2f}%")
    
    # Overfitting check
    if results['Train_R2'] - results['Test_R2'] > 0.1:
        print(f"\n Warning: Possible overfitting detected!")
        print(f"   Train R² is {results['Train_R2'] - results['Test_R2']:.4f} higher than Test R²")
    else:
        print(f"\n Model generalizes well!")

print("\n Evaluation functions defined!")

In [ ]:
# ============================================
# BASELINE MODEL 1: LINEAR REGRESSION
# ============================================

print("="*60)
print("TRAINING BASELINE MODELS")
print("="*60)

# Store all results
all_results = []

print(f"\n Training Model 1: Linear Regression...")

# Train Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Evaluate
lr_results = evaluate_model(
    lr_model, 
    X_train_scaled, 
    X_test_scaled, 
    y_train, 
    y_test, 
    model_name="Linear Regression"
)

# Print results
print_evaluation(lr_results)

# Store results
all_results.append(lr_results)

print(f"\n Linear Regression completed!")

In [ ]:
# ============================================
# BASELINE MODEL 2: RIDGE REGRESSION (L2 Regularization)
# ============================================

print("\n" + "="*60)
print("TRAINING RIDGE REGRESSION")
print("="*60)

print(f"\n Training Model 2: Ridge Regression...")

# Train Ridge Regression with default alpha
ridge_model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_model.fit(X_train_scaled, y_train)

# Evaluate
ridge_results = evaluate_model(
    ridge_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="Ridge Regression"
)

# Print results
print_evaluation(ridge_results)

# Store results
all_results.append(ridge_results)

print(f"\n Ridge Regression completed!")

In [ ]:
# ============================================
# BASELINE MODEL 3: LASSO REGRESSION (L1 Regularization)
# ============================================

print("\n" + "="*60)
print("TRAINING LASSO REGRESSION")
print("="*60)

print(f"\nTraining Model 3: Lasso Regression...")

# Train Lasso Regression with default alpha
lasso_model = Lasso(alpha=0.1, random_state=RANDOM_STATE, max_iter=5000)
lasso_model.fit(X_train_scaled, y_train)

# Evaluate
lasso_results = evaluate_model(
    lasso_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="Lasso Regression"
)

# Print results
print_evaluation(lasso_results)

# Store results
all_results.append(lasso_results)

# Check feature selection (Lasso can zero out coefficients)
non_zero_coefs = np.sum(lasso_model.coef_ != 0)
print(f"\n Feature Selection:")
print(f"   Features with non-zero coefficients: {non_zero_coefs}/{len(lasso_model.coef_)}")
print(f"   Features zeroed out: {len(lasso_model.coef_) - non_zero_coefs}")

print(f"\n Lasso Regression completed!")

In [ ]:
# ============================================
# BASELINE MODEL 4: ELASTICNET REGRESSION (L1 + L2)
# ============================================

print("\n" + "="*60)
print("TRAINING ELASTICNET REGRESSION")
print("="*60)

print(f"\n Training Model 4: ElasticNet Regression...")

# Train ElasticNet (combines Ridge and Lasso)
elasticnet_model = ElasticNet(
    alpha=0.1, 
    l1_ratio=0.5,  # 0.5 means equal mix of L1 and L2
    random_state=RANDOM_STATE,
    max_iter=5000
)
elasticnet_model.fit(X_train_scaled, y_train)

# Evaluate
elasticnet_results = evaluate_model(
    elasticnet_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="ElasticNet Regression"
)

# Print results
print_evaluation(elasticnet_results)

# Store results
all_results.append(elasticnet_results)

print(f"\n ElasticNet Regression completed!")

In [ ]:
# ============================================
# COMPARE BASELINE MODELS
# ============================================

print("\n" + "="*60)
print("BASELINE MODELS COMPARISON")
print("="*60)

# Create comparison dataframe
baseline_comparison = pd.DataFrame([
    {
        'Model': result['Model'],
        'Test_RMSE': result['Test_RMSE'],
        'Test_MAE': result['Test_MAE'],
        'Test_R2': result['Test_R2'],
        'Test_MAPE': result['Test_MAPE']
    }
    for result in all_results
])

print(f"\n Baseline Models Performance:")
display(baseline_comparison.sort_values('Test_R2', ascending=False))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Test RMSE
axes[0, 0].bar(baseline_comparison['Model'], baseline_comparison['Test_RMSE'],
              color='coral', edgecolor='black', alpha=0.7)
axes[0, 0].set_ylabel('RMSE (Lower is Better)')
axes[0, 0].set_title('Test RMSE Comparison', fontweight='bold', fontsize=12)
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(alpha=0.3, axis='y')

# Test MAE
axes[0, 1].bar(baseline_comparison['Model'], baseline_comparison['Test_MAE'],
              color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 1].set_ylabel('MAE (Lower is Better)')
axes[0, 1].set_title('Test MAE Comparison', fontweight='bold', fontsize=12)
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(alpha=0.3, axis='y')

# Test R²
axes[1, 0].bar(baseline_comparison['Model'], baseline_comparison['Test_R2'],
              color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 0].set_ylabel('R² (Higher is Better)')
axes[1, 0].set_title('Test R² Comparison', fontweight='bold', fontsize=12)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(alpha=0.3, axis='y')
axes[1, 0].axhline(y=0.5, color='red', linestyle='--', linewidth=1, label='R²=0.5')
axes[1, 0].legend()

# Test MAPE
axes[1, 1].bar(baseline_comparison['Model'], baseline_comparison['Test_MAPE'],
              color='plum', edgecolor='black', alpha=0.7)
axes[1, 1].set_ylabel('MAPE % (Lower is Better)')
axes[1, 1].set_title('Test MAPE Comparison', fontweight='bold', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Find best baseline model
best_baseline = baseline_comparison.loc[baseline_comparison['Test_R2'].idxmax()]
print(f"\n Best Baseline Model: {best_baseline['Model']}")
print(f"   Test R²: {best_baseline['Test_R2']:.4f}")
print(f"   Test RMSE: {best_baseline['Test_RMSE']:.4f}")

In [ ]:
# ============================================
# TREE-BASED MODEL 1: RANDOM FOREST
# ============================================

print("\n" + "="*60)
print("TRAINING TREE-BASED MODELS")
print("="*60)

print(f"\n Training Model 5: Random Forest Regressor...")
print(f"   (This may take a few minutes...)")

# Train Random Forest (use unscaled features for tree models)
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_train, y_train)

# Evaluate
rf_results = evaluate_model(
    rf_model,
    X_train,
    X_test,
    y_train,
    y_test,
    model_name="Random Forest"
)

# Print results
print_evaluation(rf_results)

# Store results
all_results.append(rf_results)

# Feature importance
print(f"\n Top 10 Most Important Features:")
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

for i, row in enumerate(feature_importance.head(10).itertuples(), 1):
    print(f"   {i}. {row.Feature[:50]}: {row.Importance:.4f}")

print(f"\n Random Forest completed!")

In [ ]:
# ============================================
# TREE-BASED MODEL 2: GRADIENT BOOSTING
# ============================================

print("\n" + "="*60)
print("TRAINING GRADIENT BOOSTING")
print("="*60)

print(f"\n Training Model 6: Gradient Boosting Regressor...")
print(f"   (This may take a few minutes...)")

# Train Gradient Boosting
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,
    random_state=RANDOM_STATE,
    verbose=0
)

gb_model.fit(X_train, y_train)

# Evaluate
gb_results = evaluate_model(
    gb_model,
    X_train,
    X_test,
    y_train,
    y_test,
    model_name="Gradient Boosting"
)

# Print results
print_evaluation(gb_results)

# Store results
all_results.append(gb_results)

# Feature importance
print(f"\n Top 10 Most Important Features:")
feature_importance_gb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': gb_model.feature_importances_
}).sort_values('Importance', ascending=False)

for i, row in enumerate(feature_importance_gb.head(10).itertuples(), 1):
    print(f"   {i}. {row.Feature[:50]}: {row.Importance:.4f}")

print(f"\n Gradient Boosting completed!")

In [ ]:
 # ============================================
# TREE-BASED MODEL 3: XGBOOST
# ============================================

print("\n" + "="*60)
print("TRAINING XGBOOST")
print("="*60)

print(f"\n Training Model 7: XGBoost Regressor...")
print(f"   (This may take a few minutes...)")

# Train XGBoost
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(X_train, y_train)

# Evaluate
xgb_results = evaluate_model(
    xgb_model,
    X_train,
    X_test,
    y_train,
    y_test,
    model_name="XGBoost"
)

# Print results
print_evaluation(xgb_results)

# Store results
all_results.append(xgb_results)

# Feature importance
print(f"\n Top 10 Most Important Features:")
feature_importance_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

for i, row in enumerate(feature_importance_xgb.head(10).itertuples(), 1):
    print(f"   {i}. {row.Feature[:50]}: {row.Importance:.4f}")

print(f"\n XGBoost completed!")

In [ ]:
# ============================================
# ADVANCED MODEL: LIGHTGBM
# ============================================

print("\n" + "="*60)
print("TRAINING LIGHTGBM")
print("="*60)

print(f"\n Training Model 8: LightGBM Regressor...")
print(f"   (This may take a few minutes...)")

# Train LightGBM
lgbm_model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)

lgbm_model.fit(X_train, y_train)

# Evaluate
lgbm_results = evaluate_model(
    lgbm_model,
    X_train,
    X_test,
    y_train,
    y_test,
    model_name="LightGBM"
)

# Print results
print_evaluation(lgbm_results)

# Store results
all_results.append(lgbm_results)

# Feature importance
print(f"\n Top 10 Most Important Features:")
feature_importance_lgbm = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': lgbm_model.feature_importances_
}).sort_values('Importance', ascending=False)

for i, row in enumerate(feature_importance_lgbm.head(10).itertuples(), 1):
    print(f"   {i}. {row.Feature[:50]}: {row.Importance:.4f}")

print(f"\n LightGBM completed!")

In [ ]:
# ============================================
# COMPARE ALL MODELS
# ============================================

print("\n" + "="*60)
print("ALL MODELS COMPARISON")
print("="*60)

# Create comprehensive comparison dataframe
model_comparison = pd.DataFrame([
    {
        'Model': result['Model'],
        'Train_RMSE': result['Train_RMSE'],
        'Test_RMSE': result['Test_RMSE'],
        'Train_MAE': result['Train_MAE'],
        'Test_MAE': result['Test_MAE'],
        'Train_R2': result['Train_R2'],
        'Test_R2': result['Test_R2'],
        'Train_MAPE': result['Train_MAPE'],
        'Test_MAPE': result['Test_MAPE'],
        'Overfit_Gap': result['Train_R2'] - result['Test_R2']
    }
    for result in all_results
])

# Sort by Test R² (descending)
model_comparison = model_comparison.sort_values('Test_R2', ascending=False)

print(f"\n Complete Model Performance Table:")
print(f"{'='*100}")
display(model_comparison)

# Visualize comprehensive comparison
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Test RMSE
axes[0, 0].barh(range(len(model_comparison)), model_comparison['Test_RMSE'],
               color='coral', edgecolor='black', alpha=0.7)
axes[0, 0].set_yticks(range(len(model_comparison)))
axes[0, 0].set_yticklabels(model_comparison['Model'])
axes[0, 0].set_xlabel('RMSE')
axes[0, 0].set_title('Test RMSE (Lower is Better)', fontweight='bold', fontsize=12)
axes[0, 0].invert_yaxis()
axes[0, 0].grid(alpha=0.3, axis='x')

# 2. Test R²
axes[0, 1].barh(range(len(model_comparison)), model_comparison['Test_R2'],
               color='lightgreen', edgecolor='black', alpha=0.7)
axes[0, 1].set_yticks(range(len(model_comparison)))
axes[0, 1].set_yticklabels(model_comparison['Model'])
axes[0, 1].set_xlabel('R²')
axes[0, 1].set_title('Test R² (Higher is Better)', fontweight='bold', fontsize=12)
axes[0, 1].invert_yaxis()
axes[0, 1].grid(alpha=0.3, axis='x')
axes[0, 1].axvline(x=0.8, color='red', linestyle='--', linewidth=1, label='R²=0.8')
axes[0, 1].legend()

# 3. Test MAE
axes[0, 2].barh(range(len(model_comparison)), model_comparison['Test_MAE'],
               color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 2].set_yticks(range(len(model_comparison)))
axes[0, 2].set_yticklabels(model_comparison['Model'])
axes[0, 2].set_xlabel('MAE')
axes[0, 2].set_title('Test MAE (Lower is Better)', fontweight='bold', fontsize=12)
axes[0, 2].invert_yaxis()
axes[0, 2].grid(alpha=0.3, axis='x')

# 4. Train vs Test R² (Overfitting check)
x = np.arange(len(model_comparison))
width = 0.35

axes[1, 0].bar(x - width/2, model_comparison['Train_R2'], width,
              label='Train R²', color='lightblue', edgecolor='black', alpha=0.7)
axes[1, 0].bar(x + width/2, model_comparison['Test_R2'], width,
              label='Test R²', color='orange', edgecolor='black', alpha=0.7)
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(model_comparison['Model'], rotation=45, ha='right')
axes[1, 0].set_ylabel('R²')
axes[1, 0].set_title('Train vs Test R² (Overfitting Check)', fontweight='bold', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3, axis='y')

# 5. Overfitting Gap
axes[1, 1].barh(range(len(model_comparison)), model_comparison['Overfit_Gap'],
               color='purple', edgecolor='black', alpha=0.7)
axes[1, 1].set_yticks(range(len(model_comparison)))
axes[1, 1].set_yticklabels(model_comparison['Model'])
axes[1, 1].set_xlabel('Overfitting Gap (Train R² - Test R²)')
axes[1, 1].set_title('Overfitting Analysis', fontweight='bold', fontsize=12)
axes[1, 1].invert_yaxis()
axes[1, 1].axvline(x=0.1, color='red', linestyle='--', linewidth=1, label='Gap=0.1')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3, axis='x')

# 6. Test MAPE
axes[1, 2].barh(range(len(model_comparison)), model_comparison['Test_MAPE'],
               color='plum', edgecolor='black', alpha=0.7)
axes[1, 2].set_yticks(range(len(model_comparison)))
axes[1, 2].set_yticklabels(model_comparison['Model'])
axes[1, 2].set_xlabel('MAPE (%)')
axes[1, 2].set_title('Test MAPE (Lower is Better)', fontweight='bold', fontsize=12)
axes[1, 2].invert_yaxis()
axes[1, 2].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Identify best model
best_model_idx = model_comparison['Test_R2'].idxmax()
best_model_name = model_comparison.loc[best_model_idx, 'Model']
best_test_r2 = model_comparison.loc[best_model_idx, 'Test_R2']
best_test_rmse = model_comparison.loc[best_model_idx, 'Test_RMSE']

print(f"\n" + "="*60)
print(f" BEST MODEL: {best_model_name}")
print(f"="*60)
print(f"   Test R²: {best_test_r2:.4f}")
print(f"   Test RMSE: {best_test_rmse:.4f}")
print(f"   Test MAE: {model_comparison.loc[best_model_idx, 'Test_MAE']:.4f}")
print(f"   Test MAPE: {model_comparison.loc[best_model_idx, 'Test_MAPE']:.2f}%")
print(f"   Overfitting Gap: {model_comparison.loc[best_model_idx, 'Overfit_Gap']:.4f}")

In [ ]:
# ============================================
# HYPERPARAMETER TUNING WITH OPTUNA
# ============================================

print("="*60)
print("HYPERPARAMETER TUNING WITH OPTUNA")
print("="*60)

# Install optuna if not already installed
try:
    import optuna
    from optuna.samplers import TPESampler
    print(f" Optuna version: {optuna.__version__}")
except:
    print("Installing Optuna...")
    import sys
    !{sys.executable} -m pip install optuna
    import optuna
    from optuna.samplers import TPESampler
    print(f" Optuna installed: {optuna.__version__}")

# Suppress optuna logs for cleaner output
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Get top 3 models by Test R²
top_3_models = model_comparison.head(3)['Model'].tolist()

print(f"\n Tuning top 3 models with Optuna (50 trials each):")
for i, model in enumerate(top_3_models, 1):
    print(f"   {i}. {model}")

print(f"\n Optuna will automatically find optimal hyperparameters!")

# Store tuned results
tuned_results = []
tuned_models = {}

In [ ]:
# ============================================
# OPTUNA TUNE: XGBOOST
# ============================================

if 'XGBoost' in top_3_models:
    print("\n" + "="*60)
    print("OPTUNA TUNING: XGBOOST")
    print("="*60)
    
    def objective_xgb(trial):
        """Objective function for XGBoost"""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
            'random_state': RANDOM_STATE,
            'n_jobs': -1,
            'verbosity': 0
        }
        
        # Train model
        model = XGBRegressor(**params)
        
        # Cross-validation score
        cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2', n_jobs=-1)
        
        return scores.mean()
    
    print(f"\n Running Optuna optimization (50 trials)...")
    print(f"   This will take a few minutes...")
    
    # Create study
    study_xgb = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=RANDOM_STATE)
    )
    
    # Optimize
    study_xgb.optimize(objective_xgb, n_trials=10, show_progress_bar=True)
    
    print(f"\n Optimization completed!")
    print(f"   Best CV R²: {study_xgb.best_value:.4f}")
    print(f"   Number of trials: {len(study_xgb.trials)}")
    
    print(f"\n Best hyperparameters:")
    for param, value in study_xgb.best_params.items():
        print(f"   {param}: {value}")
    
    # Train final model with best parameters
    xgb_tuned = XGBRegressor(**study_xgb.best_params, random_state=RANDOM_STATE, n_jobs=-1, verbosity=0)
    xgb_tuned.fit(X_train, y_train)
    
    # Evaluate
    xgb_tuned_results = evaluate_model(
        xgb_tuned,
        X_train,
        X_test,
        y_train,
        y_test,
        model_name="XGBoost (Optuna)"
    )
    
    print_evaluation(xgb_tuned_results)
    tuned_results.append(xgb_tuned_results)
    tuned_models['XGBoost (Optuna)'] = xgb_tuned
    
    # Visualize optimization history
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Optimization history
    trial_numbers = [t.number for t in study_xgb.trials]
    trial_values = [t.value for t in study_xgb.trials]
    
    axes[0].plot(trial_numbers, trial_values, 'o-', alpha=0.6, color='purple')
    axes[0].axhline(y=study_xgb.best_value, color='red', linestyle='--', linewidth=2, label='Best')
    axes[0].set_xlabel('Trial Number')
    axes[0].set_ylabel('CV R²')
    axes[0].set_title('XGBoost - Optimization History', fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Parameter importance
    try:
        importance = optuna.importance.get_param_importances(study_xgb)
        params = list(importance.keys())[:10]  # Top 10
        values = list(importance.values())[:10]
        
        axes[1].barh(params, values, color='plum', edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Importance')
        axes[1].set_title('XGBoost - Top 10 Parameter Importance', fontweight='bold')
        axes[1].grid(alpha=0.3, axis='x')
    except:
        axes[1].text(0.5, 0.5, 'Parameter importance not available', 
                    ha='center', va='center', transform=axes[1].transAxes)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n XGBoost (Optuna) completed!")
else:
    print("\nXGBoost not in top 3, skipping tuning")

In [ ]:
# ============================================
# OPTUNA TUNE: LIGHTGBM
# ============================================

if 'LightGBM' in top_3_models:
    print("\n" + "="*60)
    print("OPTUNA TUNING: LIGHTGBM")
    print("="*60)
    
    def objective_lgbm(trial):
        """Objective function for LightGBM"""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
            'random_state': RANDOM_STATE,
            'n_jobs': -1,
            'verbose': -1
        }
        
        # Train model
        model = LGBMRegressor(**params)
        
        # Cross-validation score
        cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2', n_jobs=-1)
        
        return scores.mean()
    
    print(f"\n Running Optuna optimization (50 trials)...")
    print(f"   This will take a few minutes...")
    
    # Create study
    study_lgbm = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=RANDOM_STATE)
    )
    
    # Optimize
    study_lgbm.optimize(objective_lgbm, n_trials=10, show_progress_bar=True)
    
    print(f"\n Optimization completed!")
    print(f"   Best CV R²: {study_lgbm.best_value:.4f}")
    print(f"   Number of trials: {len(study_lgbm.trials)}")
    
    print(f"\n Best hyperparameters:")
    for param, value in study_lgbm.best_params.items():
        print(f"   {param}: {value}")
    
    # Train final model with best parameters
    lgbm_tuned = LGBMRegressor(**study_lgbm.best_params, random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)
    lgbm_tuned.fit(X_train, y_train)
    
    # Evaluate
    lgbm_tuned_results = evaluate_model(
        lgbm_tuned,
        X_train,
        X_test,
        y_train,
        y_test,
        model_name="LightGBM (Optuna)"
    )
    
    print_evaluation(lgbm_tuned_results)
    tuned_results.append(lgbm_tuned_results)
    tuned_models['LightGBM (Optuna)'] = lgbm_tuned
    
    # Visualize optimization history
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Optimization history
    trial_numbers = [t.number for t in study_lgbm.trials]
    trial_values = [t.value for t in study_lgbm.trials]
    
    axes[0].plot(trial_numbers, trial_values, 'o-', alpha=0.6, color='teal')
    axes[0].axhline(y=study_lgbm.best_value, color='red', linestyle='--', linewidth=2, label='Best')
    axes[0].set_xlabel('Trial Number')
    axes[0].set_ylabel('CV R²')
    axes[0].set_title('LightGBM - Optimization History', fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Parameter importance
    try:
        importance = optuna.importance.get_param_importances(study_lgbm)
        params = list(importance.keys())[:10]  # Top 10
        values = list(importance.values())[:10]
        
        axes[1].barh(params, values, color='lightblue', edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Importance')
        axes[1].set_title('LightGBM - Top 10 Parameter Importance', fontweight='bold')
        axes[1].grid(alpha=0.3, axis='x')
    except:
        axes[1].text(0.5, 0.5, 'Parameter importance not available', 
                    ha='center', va='center', transform=axes[1].transAxes)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n LightGBM (Optuna) completed!")
else:
    print("\n LightGBM not in top 3, skipping tuning")

In [ ]:
# ============================================
# COMPARE ORIGINAL VS TUNED MODELS
# ============================================

print("\n" + "="*60)
print("ORIGINAL VS TUNED MODELS COMPARISON")
print("="*60)

# Combine all results (original + tuned)
all_models_results = all_results + tuned_results

# Create comparison dataframe
final_comparison = pd.DataFrame([
    {
        'Model': result['Model'],
        'Train_RMSE': result['Train_RMSE'],
        'Test_RMSE': result['Test_RMSE'],
        'Train_MAE': result['Train_MAE'],
        'Test_MAE': result['Test_MAE'],
        'Train_R2': result['Train_R2'],
        'Test_R2': result['Test_R2'],
        'Train_MAPE': result['Train_MAPE'],
        'Test_MAPE': result['Test_MAPE'],
        'Overfit_Gap': result['Train_R2'] - result['Test_R2']
    }
    for result in all_models_results
])

# Sort by Test R²
final_comparison = final_comparison.sort_values('Test_R2', ascending=False)

print(f"\n Final Model Rankings:")
print(f"{'='*100}")
display(final_comparison)

# Visualize improvement from tuning
tuned_model_names = [r['Model'] for r in tuned_results]
original_tuned_pairs = []

for tuned_name in tuned_model_names:
    # Extract original model name (remove Optuna/Tuned suffix)
    if '(Optuna)' in tuned_name:
        original_name = tuned_name.replace(' (Optuna)', '')
    elif '(Tuned)' in tuned_name:
        original_name = tuned_name.replace(' (Tuned)', '')
    else:
        original_name = tuned_name
    
    if original_name in [r['Model'] for r in all_results]:
        original_score = final_comparison[final_comparison['Model'] == original_name]['Test_R2'].values
        tuned_score = final_comparison[final_comparison['Model'] == tuned_name]['Test_R2'].values
        
        if len(original_score) > 0 and len(tuned_score) > 0:
            original_tuned_pairs.append({
                'Model': original_name,
                'Original_R2': original_score[0],
                'Tuned_R2': tuned_score[0],
                'Improvement': tuned_score[0] - original_score[0],
                'Improvement_Pct': ((tuned_score[0] - original_score[0]) / original_score[0]) * 100
            })

if len(original_tuned_pairs) > 0:
    improvement_df = pd.DataFrame(original_tuned_pairs)
    
    print(f"\nImprovement from Hyperparameter Tuning (Optuna):")
    print(f"{'='*70}")
    display(improvement_df)
    
    # Visualize improvement
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Before vs After comparison
    x = np.arange(len(improvement_df))
    width = 0.35
    
    axes[0].bar(x - width/2, improvement_df['Original_R2'], width,
               label='Original', color='lightcoral', edgecolor='black', alpha=0.7)
    axes[0].bar(x + width/2, improvement_df['Tuned_R2'], width,
               label='Optuna Tuned', color='lightgreen', edgecolor='black', alpha=0.7)
    
    axes[0].set_xlabel('Model', fontsize=12)
    axes[0].set_ylabel('Test R²', fontsize=12)
    axes[0].set_title('Original vs Optuna Tuned Models Performance', fontweight='bold', fontsize=14)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(improvement_df['Model'], rotation=45, ha='right')
    axes[0].legend(fontsize=11)
    axes[0].grid(alpha=0.3, axis='y')
    
    # Add improvement percentages
    for i, row in improvement_df.iterrows():
        improvement_pct = row['Improvement_Pct']
        y_pos = max(row['Original_R2'], row['Tuned_R2']) + 0.01
        axes[0].text(i, y_pos, f'+{improvement_pct:.2f}%',
                    ha='center', fontweight='bold', fontsize=10, color='green')
    
    # Improvement magnitude
    axes[1].barh(range(len(improvement_df)), improvement_df['Improvement_Pct'],
                color='mediumseagreen', edgecolor='black', alpha=0.7)
    axes[1].set_yticks(range(len(improvement_df)))
    axes[1].set_yticklabels(improvement_df['Model'])
    axes[1].set_xlabel('Improvement (%)', fontsize=12)
    axes[1].set_title('R² Improvement from Optuna Tuning', fontweight='bold', fontsize=14)
    axes[1].invert_yaxis()
    axes[1].grid(alpha=0.3, axis='x')
    
    # Add value labels
    for i, val in enumerate(improvement_df['Improvement_Pct']):
        axes[1].text(val + 0.1, i, f'{val:.2f}%', va='center', fontweight='bold', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n Tuning Summary:")
    print(f"   Average improvement: {improvement_df['Improvement_Pct'].mean():.2f}%")
    print(f"   Best improvement: {improvement_df['Improvement_Pct'].max():.2f}% ({improvement_df.loc[improvement_df['Improvement_Pct'].idxmax(), 'Model']})")
    print(f"   Total models tuned: {len(improvement_df)}")

# Select final best model
best_final_idx = final_comparison['Test_R2'].idxmax()
best_final_model_name = final_comparison.loc[best_final_idx, 'Model']
best_final_r2 = final_comparison.loc[best_final_idx, 'Test_R2']
best_final_rmse = final_comparison.loc[best_final_idx, 'Test_RMSE']

print(f"\n" + "="*70)
print(f" FINAL BEST MODEL: {best_final_model_name}")
print(f"="*70)
print(f"   Test R²: {best_final_r2:.4f}")
print(f"   Test RMSE: {best_final_rmse:.4f}")
print(f"   Test MAE: {final_comparison.loc[best_final_idx, 'Test_MAE']:.4f}")
print(f"   Test MAPE: {final_comparison.loc[best_final_idx, 'Test_MAPE']:.2f}%")
print(f"   Overfitting Gap: {final_comparison.loc[best_final_idx, 'Overfit_Gap']:.4f}")

# Get the actual best model object
best_model = None

# Check if it's a tuned model
if best_final_model_name in tuned_models:
    best_model = tuned_models[best_final_model_name]
    print(f"\n Best model retrieved from tuned models")
else:
    # It's an original model
    if 'Random Forest' in best_final_model_name:
        best_model = rf_model
    elif 'Gradient Boosting' in best_final_model_name:
        best_model = gb_model
    elif 'XGBoost' in best_final_model_name:
        best_model = xgb_model
    elif 'LightGBM' in best_final_model_name:
        best_model = lgbm_model
    elif 'Linear Regression' in best_final_model_name:
        best_model = lr_model
    elif 'Ridge' in best_final_model_name:
        best_model = ridge_model
    elif 'Lasso' in best_final_model_name:
        best_model = lasso_model
    elif 'ElasticNet' in best_final_model_name:
        best_model = elasticnet_model
    
    if best_model is not None:
        print(f" Best model retrieved from original models")

if best_model is None:
    print(f" Warning: Could not retrieve best model object")

In [ ]:
# ============================================
# CROSS-VALIDATION ANALYSIS
# ============================================

print("\n" + "="*60)
print("CROSS-VALIDATION ANALYSIS")
print("="*60)

print(f"\n Performing 5-Fold Cross-Validation on final best model...")
print(f"   Model: {best_final_model_name}")

# Determine if we should use scaled or unscaled features
use_scaled = any(keyword in best_final_model_name.lower() 
                for keyword in ['linear', 'ridge', 'lasso', 'elastic'])

if use_scaled:
    X_cv = X_train_scaled
    print(f"   Using scaled features (linear model)")
else:
    X_cv = X_train
    print(f"   Using unscaled features (tree-based model)")

if best_model is not None:
    
    # Perform cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    # Calculate multiple metrics
    scoring = {
        'r2': 'r2',
        'neg_rmse': 'neg_root_mean_squared_error',
        'neg_mae': 'neg_mean_absolute_error'
    }
    
    print(f"\n   Running cross-validation...")
    cv_results = cross_validate(
        best_model,
        X_cv,
        y_train,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        verbose=0,
        return_train_score=True
    )
    
    # Extract scores
    train_r2_scores = cv_results['train_r2']
    test_r2_scores = cv_results['test_r2']
    train_rmse_scores = -cv_results['train_neg_rmse']
    test_rmse_scores = -cv_results['test_neg_rmse']
    train_mae_scores = -cv_results['train_neg_mae']
    test_mae_scores = -cv_results['test_neg_mae']
    
    print(f"\n Cross-Validation Results:")
    print(f"{'='*70}")
    
    print(f"\n R² Scores across 5 folds:")
    print(f"   {'Fold':<10} {'Train R²':<15} {'Test R²':<15} {'Gap':<10}")
    print(f"   {'-'*50}")
    for i in range(5):
        gap = train_r2_scores[i] - test_r2_scores[i]
        print(f"   Fold {i+1:<5} {train_r2_scores[i]:<15.4f} {test_r2_scores[i]:<15.4f} {gap:<10.4f}")
    
    print(f"\n   Mean Train R²: {train_r2_scores.mean():.4f} (+/- {train_r2_scores.std() * 2:.4f})")
    print(f"   Mean Test R²:  {test_r2_scores.mean():.4f} (+/- {test_r2_scores.std() * 2:.4f})")
    print(f"   Mean Gap:      {(train_r2_scores.mean() - test_r2_scores.mean()):.4f}")
    
    print(f"\nRMSE Scores across 5 folds:")
    print(f"   {'Fold':<10} {'Train RMSE':<15} {'Test RMSE':<15}")
    print(f"   {'-'*50}")
    for i in range(5):
        print(f"   Fold {i+1:<5} {train_rmse_scores[i]:<15.4f} {test_rmse_scores[i]:<15.4f}")
    
    print(f"\n   Mean Train RMSE: {train_rmse_scores.mean():.4f} (+/- {train_rmse_scores.std() * 2:.4f})")
    print(f"   Mean Test RMSE:  {test_rmse_scores.mean():.4f} (+/- {test_rmse_scores.std() * 2:.4f})")
    
    print(f"\n MAE Scores across 5 folds:")
    print(f"   {'Fold':<10} {'Train MAE':<15} {'Test MAE':<15}")
    print(f"   {'-'*50}")
    for i in range(5):
        print(f"   Fold {i+1:<5} {train_mae_scores[i]:<15.4f} {test_mae_scores[i]:<15.4f}")
    
    print(f"\n   Mean Train MAE: {train_mae_scores.mean():.4f} (+/- {train_mae_scores.std() * 2:.4f})")
    print(f"   Mean Test MAE:  {test_mae_scores.mean():.4f} (+/- {test_mae_scores.std() * 2:.4f})")
    
    # Visualize CV results
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. R² Train vs Test
    fold_numbers = np.arange(1, 6)
    axes[0, 0].plot(fold_numbers, train_r2_scores, 'o-', label='Train R²', 
                   linewidth=2, markersize=8, color='blue')
    axes[0, 0].plot(fold_numbers, test_r2_scores, 's-', label='Test R²', 
                   linewidth=2, markersize=8, color='orange')
    axes[0, 0].axhline(y=train_r2_scores.mean(), color='blue', linestyle='--', alpha=0.5)
    axes[0, 0].axhline(y=test_r2_scores.mean(), color='orange', linestyle='--', alpha=0.5)
    axes[0, 0].set_xlabel('Fold Number', fontsize=11)
    axes[0, 0].set_ylabel('R² Score', fontsize=11)
    axes[0, 0].set_title('R² Scores Across Folds', fontweight='bold', fontsize=12)
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    axes[0, 0].set_xticks(fold_numbers)
    
    # 2. R² Box plot
    axes[0, 1].boxplot([train_r2_scores, test_r2_scores], 
                      labels=['Train', 'Test'],
                      patch_artist=True,
                      boxprops=dict(facecolor='lightblue', alpha=0.7))
    axes[0, 1].scatter([1]*5, train_r2_scores, alpha=0.6, s=100, color='blue', zorder=3)
    axes[0, 1].scatter([2]*5, test_r2_scores, alpha=0.6, s=100, color='orange', zorder=3)
    axes[0, 1].set_ylabel('R² Score', fontsize=11)
    axes[0, 1].set_title('R² Distribution', fontweight='bold', fontsize=12)
    axes[0, 1].grid(alpha=0.3, axis='y')
    
    # 3. RMSE Train vs Test
    axes[0, 2].plot(fold_numbers, train_rmse_scores, 'o-', label='Train RMSE', 
                   linewidth=2, markersize=8, color='green')
    axes[0, 2].plot(fold_numbers, test_rmse_scores, 's-', label='Test RMSE', 
                   linewidth=2, markersize=8, color='red')
    axes[0, 2].axhline(y=train_rmse_scores.mean(), color='green', linestyle='--', alpha=0.5)
    axes[0, 2].axhline(y=test_rmse_scores.mean(), color='red', linestyle='--', alpha=0.5)
    axes[0, 2].set_xlabel('Fold Number', fontsize=11)
    axes[0, 2].set_ylabel('RMSE', fontsize=11)
    axes[0, 2].set_title('RMSE Across Folds', fontweight='bold', fontsize=12)
    axes[0, 2].legend()
    axes[0, 2].grid(alpha=0.3)
    axes[0, 2].set_xticks(fold_numbers)
    
    # 4. RMSE Box plot
    axes[1, 0].boxplot([train_rmse_scores, test_rmse_scores], 
                      labels=['Train', 'Test'],
                      patch_artist=True,
                      boxprops=dict(facecolor='lightcoral', alpha=0.7))
    axes[1, 0].scatter([1]*5, train_rmse_scores, alpha=0.6, s=100, color='green', zorder=3)
    axes[1, 0].scatter([2]*5, test_rmse_scores, alpha=0.6, s=100, color='red', zorder=3)
    axes[1, 0].set_ylabel('RMSE', fontsize=11)
    axes[1, 0].set_title('RMSE Distribution', fontweight='bold', fontsize=12)
    axes[1, 0].grid(alpha=0.3, axis='y')
    
    # 5. MAE Train vs Test
    axes[1, 1].plot(fold_numbers, train_mae_scores, 'o-', label='Train MAE', 
                   linewidth=2, markersize=8, color='purple')
    axes[1, 1].plot(fold_numbers, test_mae_scores, 's-', label='Test MAE', 
                   linewidth=2, markersize=8, color='brown')
    axes[1, 1].axhline(y=train_mae_scores.mean(), color='purple', linestyle='--', alpha=0.5)
    axes[1, 1].axhline(y=test_mae_scores.mean(), color='brown', linestyle='--', alpha=0.5)
    axes[1, 1].set_xlabel('Fold Number', fontsize=11)
    axes[1, 1].set_ylabel('MAE', fontsize=11)
    axes[1, 1].set_title('MAE Across Folds', fontweight='bold', fontsize=12)
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    axes[1, 1].set_xticks(fold_numbers)
    
    # 6. MAE Box plot
    axes[1, 2].boxplot([train_mae_scores, test_mae_scores], 
                      labels=['Train', 'Test'],
                      patch_artist=True,
                      boxprops=dict(facecolor='lightgreen', alpha=0.7))
    axes[1, 2].scatter([1]*5, train_mae_scores, alpha=0.6, s=100, color='purple', zorder=3)
    axes[1, 2].scatter([2]*5, test_mae_scores, alpha=0.6, s=100, color='brown', zorder=3)
    axes[1, 2].set_ylabel('MAE', fontsize=11)
    axes[1, 2].set_title('MAE Distribution', fontweight='bold', fontsize=12)
    axes[1, 2].grid(alpha=0.3, axis='y')
    
    plt.suptitle(f'Cross-Validation Analysis - {best_final_model_name}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    # Stability assessment
    r2_std = test_r2_scores.std()
    rmse_std = test_rmse_scores.std()
    
    print(f"\n Model Stability Assessment:")
    print(f"{'='*60}")
    print(f"   Test R² std: {r2_std:.4f}")
    print(f"   Test RMSE std: {rmse_std:.4f}")
    
    if r2_std < 0.05:
        print(f"\n    Model is very stable! (R² std < 0.05)")
    elif r2_std < 0.1:
        print(f"\n    Model is reasonably stable (R² std < 0.1)")
    else:
        print(f"\n    Model shows high variance across folds (R² std >= 0.1)")
        print(f"      Consider collecting more data or adjusting regularization")
    
    # Overfitting check
    avg_gap = (train_r2_scores.mean() - test_r2_scores.mean())
    print(f"\n Overfitting Check:")
    print(f"   Average Train-Test Gap: {avg_gap:.4f}")
    
    if avg_gap < 0.05:
        print(f"   Excellent! Model generalizes very well")
    elif avg_gap < 0.1:
        print(f"    Good! Model generalizes well")
    elif avg_gap < 0.15:
        print(f"    Slight overfitting detected")
    else:
        print(f"    Significant overfitting detected - consider regularization")
    
    print(f"\n Cross-validation analysis completed!")
    
else:
    print(f"\n Could not find best model object for CV analysis")

In [ ]:
# ============================================
# RESIDUAL ANALYSIS
# ============================================

print("\n" + "="*60)
print("RESIDUAL ANALYSIS - BEST MODEL")
print("="*60)

print(f"\n Analyzing residuals for: {best_final_model_name}")

# Get predictions from best model
best_result = None
for result in all_models_results:
    if result['Model'] == best_final_model_name:
        best_result = result
        break

if best_result is not None:
    y_train_pred = best_result['Predictions_Train']
    y_test_pred = best_result['Predictions_Test']
    
    # Calculate residuals
    train_residuals = y_train - y_train_pred
    test_residuals = y_test - y_test_pred
    
    print(f"\n Residual Statistics:")
    print(f"{'='*60}")
    print(f"Training Residuals:")
    print(f"   Mean: {train_residuals.mean():.4f}")
    print(f"   Std: {train_residuals.std():.4f}")
    print(f"   Min: {train_residuals.min():.4f}")
    print(f"   Max: {train_residuals.max():.4f}")
    
    print(f"\nTest Residuals:")
    print(f"   Mean: {test_residuals.mean():.4f}")
    print(f"   Std: {test_residuals.std():.4f}")
    print(f"   Min: {test_residuals.min():.4f}")
    print(f"   Max: {test_residuals.max():.4f}")
    
    # Create comprehensive residual plots
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. Residuals vs Predicted (Train)
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.scatter(y_train_pred, train_residuals, alpha=0.5, s=20, color='blue')
    ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Residuals')
    ax1.set_title('Train: Residuals vs Predicted', fontweight='bold')
    ax1.grid(alpha=0.3)
    
    # 2. Residuals vs Predicted (Test)
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.scatter(y_test_pred, test_residuals, alpha=0.5, s=20, color='orange')
    ax2.axhline(y=0, color='red', linestyle='--', linewidth=2)
    ax2.set_xlabel('Predicted Values')
    ax2.set_ylabel('Residuals')
    ax2.set_title('Test: Residuals vs Predicted', fontweight='bold')
    ax2.grid(alpha=0.3)
    
    # 3. Distribution of Residuals (Train)
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.hist(train_residuals, bins=50, edgecolor='black', alpha=0.7, color='blue')
    ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax3.set_xlabel('Residuals')
    ax3.set_ylabel('Frequency')
    ax3.set_title('Train: Residual Distribution', fontweight='bold')
    ax3.grid(alpha=0.3)
    
    # 4. Distribution of Residuals (Test)
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.hist(test_residuals, bins=50, edgecolor='black', alpha=0.7, color='orange')
    ax4.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax4.set_xlabel('Residuals')
    ax4.set_ylabel('Frequency')
    ax4.set_title('Test: Residual Distribution', fontweight='bold')
    ax4.grid(alpha=0.3)
    
    # 5. Q-Q Plot (Train)
    ax5 = fig.add_subplot(gs[1, 1])
    stats.probplot(train_residuals, dist="norm", plot=ax5)
    ax5.set_title('Train: Q-Q Plot', fontweight='bold')
    ax5.grid(alpha=0.3)
    
    # 6. Q-Q Plot (Test)
    ax6 = fig.add_subplot(gs[1, 2])
    stats.probplot(test_residuals, dist="norm", plot=ax6)
    ax6.set_title('Test: Q-Q Plot', fontweight='bold')
    ax6.grid(alpha=0.3)
    
    # 7. Actual vs Predicted (Train)
    ax7 = fig.add_subplot(gs[2, 0])
    ax7.scatter(y_train, y_train_pred, alpha=0.5, s=20, color='blue')
    
    # Perfect prediction line
    min_val = min(y_train.min(), y_train_pred.min())
    max_val = max(y_train.max(), y_train_pred.max())
    ax7.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    ax7.set_xlabel('Actual Values')
    ax7.set_ylabel('Predicted Values')
    ax7.set_title('Train: Actual vs Predicted', fontweight='bold')
    ax7.legend()
    ax7.grid(alpha=0.3)
    
    # 8. Actual vs Predicted (Test)
    ax8 = fig.add_subplot(gs[2, 1])
    ax8.scatter(y_test, y_test_pred, alpha=0.5, s=20, color='orange')
    
    # Perfect prediction line
    min_val = min(y_test.min(), y_test_pred.min())
    max_val = max(y_test.max(), y_test_pred.max())
    ax8.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    ax8.set_xlabel('Actual Values')
    ax8.set_ylabel('Predicted Values')
    ax8.set_title('Test: Actual vs Predicted', fontweight='bold')
    ax8.legend()
    ax8.grid(alpha=0.3)
    
    # 9. Residual Box Plot Comparison
    ax9 = fig.add_subplot(gs[2, 2])
    ax9.boxplot([train_residuals, test_residuals], 
                labels=['Train', 'Test'],
                patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7))
    ax9.axhline(y=0, color='red', linestyle='--', linewidth=2)
    ax9.set_ylabel('Residuals')
    ax9.set_title('Residuals Comparison', fontweight='bold')
    ax9.grid(alpha=0.3)
    
    plt.suptitle(f'Residual Analysis - {best_final_model_name}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.show()
    
    # Statistical tests
    print(f"\n Statistical Tests:")
    print(f"{'='*60}")
    
    # Shapiro-Wilk test for normality (sample if too large)
    sample_size = min(5000, len(test_residuals))
    test_residuals_sample = np.random.choice(test_residuals, size=sample_size, replace=False)
    
    shapiro_stat, shapiro_p = stats.shapiro(test_residuals_sample)
    print(f"Shapiro-Wilk Test (Normality of Residuals):")
    print(f"   Statistic: {shapiro_stat:.4f}")
    print(f"   P-value: {shapiro_p:.4f}")
    
    if shapiro_p > 0.05:
        print(f"    Residuals are approximately normally distributed (p > 0.05)")
    else:
        print(f"    Residuals may not be normally distributed (p < 0.05)")
    
    # Check for patterns (randomness)
    print(f"\n Residual Pattern Check:")
    
    # Calculate correlation between predictions and residuals
    corr_train = np.corrcoef(y_train_pred, train_residuals)[0, 1]
    corr_test = np.corrcoef(y_test_pred, test_residuals)[0, 1]
    
    print(f"   Correlation (Predictions vs Residuals):")
    print(f"      Train: {corr_train:.4f}")
    print(f"      Test: {corr_test:.4f}")
    
    if abs(corr_test) < 0.1:
        print(f"    Residuals show no pattern (good!)")
    else:
        print(f"    Residuals show some pattern (model may miss non-linear relationships)")
    
    print(f"\n Residual analysis completed!")
    
else:
    print(f"\n Could not find best model results for residual analysis")

In [ ]:
# ============================================
# PREDICTION VISUALIZATION (ORIGINAL SCALE)
# ============================================

print("\n" + "="*60)
print("PREDICTION VISUALIZATION - ORIGINAL SCALE")
print("="*60)

print(f"\n Converting log-transformed predictions back to original PM2.5 scale...")

# Convert predictions back to original scale
# Remember: we used log1p, so we use expm1 to reverse
y_train_pred_original = np.expm1(y_train_pred)
y_test_pred_original = np.expm1(y_test_pred)

# Calculate metrics on original scale
train_rmse_original = np.sqrt(mean_squared_error(y_train_original, y_train_pred_original))
test_rmse_original = np.sqrt(mean_squared_error(y_test_original, y_test_pred_original))

train_mae_original = mean_absolute_error(y_train_original, y_train_pred_original)
test_mae_original = mean_absolute_error(y_test_original, y_test_pred_original)

train_r2_original = r2_score(y_train_original, y_train_pred_original)
test_r2_original = r2_score(y_test_original, y_test_pred_original)

train_mape_original = np.mean(np.abs((y_train_original - y_train_pred_original) / (y_train_original + 1e-10))) * 100
test_mape_original = np.mean(np.abs((y_test_original - y_test_pred_original) / (y_test_original + 1e-10))) * 100

print(f"\n Performance on Original PM2.5 Scale (μg/m³):")
print(f"{'='*60}")
print(f"Training Set:")
print(f"   RMSE: {train_rmse_original:.2f} μg/m³")
print(f"   MAE: {train_mae_original:.2f} μg/m³")
print(f"   R²: {train_r2_original:.4f}")
print(f"   MAPE: {train_mape_original:.2f}%")

print(f"\nTest Set:")
print(f"   RMSE: {test_rmse_original:.2f} μg/m³")
print(f"   MAE: {test_mae_original:.2f} μg/m³")
print(f"   R²: {test_r2_original:.4f}")
print(f"   MAPE: {test_mape_original:.2f}%")

# Visualizations on original scale
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Actual vs Predicted - Train
axes[0, 0].scatter(y_train_original, y_train_pred_original, alpha=0.5, s=20, color='blue')
min_val = min(y_train_original.min(), y_train_pred_original.min())
max_val = max(y_train_original.max(), y_train_pred_original.max())
axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual PM2.5 (μg/m³)')
axes[0, 0].set_ylabel('Predicted PM2.5 (μg/m³)')
axes[0, 0].set_title(f'Train: Actual vs Predicted\nR² = {train_r2_original:.4f}', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Actual vs Predicted - Test
axes[0, 1].scatter(y_test_original, y_test_pred_original, alpha=0.5, s=20, color='orange')
min_val = min(y_test_original.min(), y_test_pred_original.min())
max_val = max(y_test_original.max(), y_test_pred_original.max())
axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual PM2.5 (μg/m³)')
axes[0, 1].set_ylabel('Predicted PM2.5 (μg/m³)')
axes[0, 1].set_title(f'Test: Actual vs Predicted\nR² = {test_r2_original:.4f}', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. Prediction Error Distribution - Train
train_errors = y_train_original - y_train_pred_original
axes[0, 2].hist(train_errors, bins=50, edgecolor='black', alpha=0.7, color='blue')
axes[0, 2].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 2].set_xlabel('Prediction Error (μg/m³)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title(f'Train: Error Distribution\nMAE = {train_mae_original:.2f} μg/m³', fontweight='bold')
axes[0, 2].grid(alpha=0.3)

# 4. Prediction Error Distribution - Test
test_errors = y_test_original - y_test_pred_original
axes[1, 0].hist(test_errors, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Prediction Error (μg/m³)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'Test: Error Distribution\nMAE = {test_mae_original:.2f} μg/m³', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 5. Sample Predictions Comparison - Test (first 100)
sample_size = min(100, len(y_test_original))
sample_idx = np.arange(sample_size)

axes[1, 1].plot(sample_idx, y_test_original[:sample_size], 'o-', label='Actual', alpha=0.7, linewidth=2, markersize=4)
axes[1, 1].plot(sample_idx, y_test_pred_original[:sample_size], 's-', label='Predicted', alpha=0.7, linewidth=2, markersize=4)
axes[1, 1].set_xlabel('Sample Index')
axes[1, 1].set_ylabel('PM2.5 (μg/m³)')
axes[1, 1].set_title('Test: Sample Predictions (First 100)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

# 6. Percentage Error Distribution - Test
percentage_errors = ((y_test_original - y_test_pred_original) / (y_test_original + 1e-10)) * 100
axes[1, 2].hist(percentage_errors, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1, 2].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 2].set_xlabel('Percentage Error (%)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title(f'Test: Percentage Error\nMAPE = {test_mape_original:.2f}%', fontweight='bold')
axes[1, 2].grid(alpha=0.3)

plt.suptitle(f'Prediction Analysis - {best_final_model_name} (Original PM2.5 Scale)', 
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# Prediction quality by PM2.5 ranges
print(f"\n Prediction Quality by PM2.5 Ranges:")
print(f"{'='*60}")

# Define PM2.5 ranges (EPA Air Quality Index)
ranges = [
    (0, 12, 'Good'),
    (12, 35, 'Moderate'),
    (35, 55, 'Unhealthy for Sensitive'),
    (55, 150, 'Unhealthy'),
    (150, 250, 'Very Unhealthy'),
    (250, 1000, 'Hazardous')
]

for low, high, label in ranges:
    mask = (y_test_original >= low) & (y_test_original < high)
    count = mask.sum()
    
    if count > 0:
        range_mae = mean_absolute_error(y_test_original[mask], y_test_pred_original[mask])
        range_mape = np.mean(np.abs((y_test_original[mask] - y_test_pred_original[mask]) / (y_test_original[mask] + 1e-10))) * 100
        
        print(f"\n{label} ({low}-{high} μg/m³):")
        print(f"   Samples: {count}")
        print(f"   MAE: {range_mae:.2f} μg/m³")
        print(f"   MAPE: {range_mape:.2f}%")

print(f"\n Prediction visualization completed!")

In [ ]:
# ============================================
# FEATURE IMPORTANCE ANALYSIS
# ============================================

print("\n" + "="*60)
print("FEATURE IMPORTANCE - BEST MODEL")
print("="*60)

print(f"\n Analyzing feature importance for: {best_final_model_name}")

# Get feature importance from best model
if best_model is not None and hasattr(best_model, 'feature_importances_'):
    
    # Get feature importances
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(f"\n Top 30 Most Important Features:")
    print(f"{'='*70}")
    for i, row in enumerate(feature_importance.head(30).itertuples(), 1):
        print(f"   {i:2d}. {row.Feature[:55]:<55} {row.Importance:.6f}")
    

    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(20, 14))
    
    # 1. Top 20 features
    top_20 = feature_importance.head(20)
    axes[0, 0].barh(range(len(top_20)), top_20['Importance'], color='steelblue', edgecolor='black', alpha=0.7)
    axes[0, 0].set_yticks(range(len(top_20)))
    axes[0, 0].set_yticklabels([f[:40] + '...' if len(f) > 40 else f for f in top_20['Feature']], fontsize=9)
    axes[0, 0].set_xlabel('Importance', fontsize=11)
    axes[0, 0].set_title('Top 20 Most Important Features', fontweight='bold', fontsize=13)
    axes[0, 0].invert_yaxis()
    axes[0, 0].grid(alpha=0.3, axis='x')
    
    # 2. Cumulative importance
    cumulative_importance = feature_importance['Importance'].cumsum() / feature_importance['Importance'].sum()
    axes[0, 1].plot(range(1, len(cumulative_importance) + 1), cumulative_importance, linewidth=2, color='darkgreen')
    axes[0, 1].axhline(y=0.8, color='red', linestyle='--', linewidth=2, label='80% threshold')
    axes[0, 1].axhline(y=0.9, color='orange', linestyle='--', linewidth=2, label='90% threshold')
    axes[0, 1].set_xlabel('Number of Features', fontsize=11)
    axes[0, 1].set_ylabel('Cumulative Importance', fontsize=11)
    axes[0, 1].set_title('Cumulative Feature Importance', fontweight='bold', fontsize=13)
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Find features needed for 80% and 90%
    features_80 = (cumulative_importance >= 0.8).argmax() + 1
    features_90 = (cumulative_importance >= 0.9).argmax() + 1
    
    axes[0, 1].axvline(x=features_80, color='red', linestyle=':', linewidth=1.5, alpha=0.7)
    axes[0, 1].axvline(x=features_90, color='orange', linestyle=':', linewidth=1.5, alpha=0.7)
    
    print(f"\n Feature Importance Statistics:")
    print(f"   Features for 80% importance: {features_80}/{len(feature_importance)}")
    print(f"   Features for 90% importance: {features_90}/{len(feature_importance)}")
    
    # 3. Feature categories importance
    feature_categories_importance = {
        'Time Series': 0,
        'Temporal': 0,
        'Weather': 0,
        'Pollutants': 0,
        'Location': 0,
        'Wind': 0,
        'Interactions': 0,
        'Other': 0
    }
    
    for _, row in feature_importance.iterrows():
        feature = row['Feature']
        importance = row['Importance']
        
        if 'rolling' in feature or 'lag' in feature or 'diff' in feature:
            feature_categories_importance['Time Series'] += importance
        elif 'month' in feature or 'day' in feature or 'season' in feature:
            feature_categories_importance['Temporal'] += importance
        elif 'temperature' in feature or 'humidity' in feature or 'precipitable' in feature:
            feature_categories_importance['Weather'] += importance
        elif 'L3_' in feature and any(x in feature for x in ['NO2', 'CO', 'SO2', 'HCHO', 'O3']):
            feature_categories_importance['Pollutants'] += importance
        elif 'place_' in feature:
            feature_categories_importance['Location'] += importance
        elif 'wind' in feature.lower():
            feature_categories_importance['Wind'] += importance
        elif 'interaction' in feature or 'pollutant_load' in feature or 'AQI' in feature or 'ratio' in feature:
            feature_categories_importance['Interactions'] += importance
        else:
            feature_categories_importance['Other'] += importance
    
    # Sort categories
    categories_sorted = dict(sorted(feature_categories_importance.items(), key=lambda x: x[1], reverse=True))
    
    axes[1, 0].bar(range(len(categories_sorted)), list(categories_sorted.values()),
                  color='coral', edgecolor='black', alpha=0.7)
    axes[1, 0].set_xticks(range(len(categories_sorted)))
    axes[1, 0].set_xticklabels(list(categories_sorted.keys()), rotation=45, ha='right')
    axes[1, 0].set_ylabel('Total Importance', fontsize=11)
    axes[1, 0].set_title('Feature Category Importance', fontweight='bold', fontsize=13)
    axes[1, 0].grid(alpha=0.3, axis='y')
    
    # Add values on bars
    for i, (cat, val) in enumerate(categories_sorted.items()):
        axes[1, 0].text(i, val + 0.01, f'{val:.3f}', ha='center', fontweight='bold', fontsize=9)
    
    print(f"\n Feature Category Importance:")
    for category, importance in categories_sorted.items():
        print(f"   {category:<20}: {importance:.4f}")
    
    # 4. Top features breakdown
    top_10 = feature_importance.head(10)
    colors = plt.cm.viridis(np.linspace(0, 1, len(top_10)))
    
    axes[1, 1].barh(range(len(top_10)), top_10['Importance'], color=colors, edgecolor='black', alpha=0.8)
    axes[1, 1].set_yticks(range(len(top_10)))
    axes[1, 1].set_yticklabels([f[:35] + '...' if len(f) > 35 else f for f in top_10['Feature']], fontsize=10)
    axes[1, 1].set_xlabel('Importance', fontsize=11)
    axes[1, 1].set_title('Top 10 Features (Detailed)', fontweight='bold', fontsize=13)
    axes[1, 1].invert_yaxis()
    axes[1, 1].grid(alpha=0.3, axis='x')
    
    # Add percentage labels
    total_importance = feature_importance['Importance'].sum()
    for i, (idx, row) in enumerate(top_10.iterrows()):
        pct = (row['Importance'] / total_importance) * 100
        axes[1, 1].text(row['Importance'] + 0.002, i, f'{pct:.1f}%', 
                       va='center', fontweight='bold', fontsize=9)
    
    plt.suptitle(f'Feature Importance Analysis - {best_final_model_name}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    print(f"\n Feature importance analysis completed!")
    
else:
    print(f"\n Best model does not have feature_importances_ attribute")
    print(f"   (Linear models don't provide feature importance in the same way)")

In [ ]:
# ============================================
# SAVE BEST MODEL AND ARTIFACTS
# ============================================

print("="*60)
print("SAVING BEST MODEL AND ARTIFACTS")
print("="*60)

import pickle
import json
import os
from datetime import datetime

# Create directories if they don't exist
os.makedirs('../artifacts/models', exist_ok=True)
os.makedirs('../artifacts/scalers', exist_ok=True)
os.makedirs('../artifacts/reports', exist_ok=True)

print(f"\n Saving model artifacts...")

# 1. Save the best model
if best_model is not None:
    model_filename = f'../artifacts/models/best_model_{best_final_model_name.replace(" ", "_").replace("(", "").replace(")", "")}.pkl'
    
    with open(model_filename, 'wb') as f:
        pickle.dump(best_model, f)
    
    print(f"   Best model saved: {model_filename}")
    print(f"      Model: {best_final_model_name}")
else:
    print(f"   Warning: Best model object not found, could not save model")

# 2. Save the scaler (for preprocessing)
scaler_filename = '../artifacts/scalers/standard_scaler.pkl'

with open(scaler_filename, 'wb') as f:
    pickle.dump(scaler, f)

print(f"    Scaler saved: {scaler_filename}")

# 3. Save feature names
feature_names_file = '../artifacts/models/feature_names.txt'

with open(feature_names_file, 'w') as f:
    for feature in X_train.columns:
        f.write(f"{feature}\n")

print(f"    Feature names saved: {feature_names_file}")

# 4. Save model comparison results
comparison_file = '../artifacts/reports/model_comparison.csv'
final_comparison.to_csv(comparison_file, index=False)

print(f"    Model comparison saved: {comparison_file}")

# 5. Save predictions
predictions_train_file = '../artifacts/reports/predictions_train.csv'
predictions_test_file = '../artifacts/reports/predictions_test.csv'

# Create prediction dataframes
if best_result is not None:
    # Training predictions
    predictions_train_df = pd.DataFrame({
        'Actual_Log': y_train,
        'Predicted_Log': y_train_pred,
        'Actual_Original': y_train_original,
        'Predicted_Original': y_train_pred_original,
        'Error': y_train_original - y_train_pred_original,
        'Absolute_Error': np.abs(y_train_original - y_train_pred_original),
        'Percentage_Error': ((y_train_original - y_train_pred_original) / (y_train_original + 1e-10)) * 100
    })
    
    # Add metadata if available
    if len(metadata_train) == len(predictions_train_df):
        predictions_train_df = pd.concat([metadata_train.reset_index(drop=True), predictions_train_df], axis=1)
    
    predictions_train_df.to_csv(predictions_train_file, index=False)
    print(f"    Training predictions saved: {predictions_train_file}")
    
    # Test predictions
    predictions_test_df = pd.DataFrame({
        'Actual_Log': y_test,
        'Predicted_Log': y_test_pred,
        'Actual_Original': y_test_original,
        'Predicted_Original': y_test_pred_original,
        'Error': y_test_original - y_test_pred_original,
        'Absolute_Error': np.abs(y_test_original - y_test_pred_original),
        'Percentage_Error': ((y_test_original - y_test_pred_original) / (y_test_original + 1e-10)) * 100
    })
    
    # Add metadata if available
    if len(metadata_test) == len(predictions_test_df):
        predictions_test_df = pd.concat([metadata_test.reset_index(drop=True), predictions_test_df], axis=1)
    
    predictions_test_df.to_csv(predictions_test_file, index=False)
    print(f"   Test predictions saved: {predictions_test_file}")


In [ ]:
# ============================================
# EXTRACT FEATURE MEDIANS FOR DEPLOYMENT
# ============================================

print("="*70)
print("EXTRACTING FEATURE MEDIANS FOR DEPLOYMENT")
print("="*70)

import json
import os
from datetime import datetime

# Create directory if it doesn't exist
os.makedirs('../artifacts/feature_engineering', exist_ok=True)

print(f"\n Analyzing training features...")
print(f"   Total features: {len(X_train.columns)}")

# ============================================
# IDENTIFY UNCALCULABLE FEATURES
# ============================================

# Features that CANNOT be calculated from single raw input
time_series_features = [col for col in X_train.columns if any(x in col for x in ['rolling', 'lag', 'diff', 'ewm', 'shift'])]
location_features = [col for col in X_train.columns if 'place_' in col and 'place_id' not in col.lower()]
pca_features = [col for col in X_train.columns if 'pca' in col.lower()]
cluster_features = [col for col in X_train.columns if 'cluster' in col]
historical_agg_features = [col for col in X_train.columns if any(x in col for x in ['_mean_', '_median_', '_std_', '_min_', '_max_'])]

# Combine all uncalculable features
uncalculable_features = list(set(
    time_series_features + 
    location_features + 
    pca_features + 
    cluster_features + 
    historical_agg_features
))

print(f"\n Feature Categories:")
print(f"   - Time series (rolling, lag, diff): {len(time_series_features)}")
print(f"   - Location aggregations: {len(location_features)}")
print(f"   - PCA components: {len(pca_features)}")
print(f"   - Cluster features: {len(cluster_features)}")
print(f"   - Historical aggregations: {len(historical_agg_features)}")
print(f"   - Total UNCALCULABLE: {len(uncalculable_features)}")
print(f"   - Total CALCULABLE: {len(X_train.columns) - len(uncalculable_features)}")

# ============================================
# CALCULATE MEDIANS
# ============================================

print(f"\n Calculating medians...")

# Medians for uncalculable features only
uncalculable_medians = {}
for feature in uncalculable_features:
    if feature in X_train.columns:
        median_value = float(X_train[feature].median())
        uncalculable_medians[feature] = median_value

# Medians for ALL features (backup)
all_feature_medians = {col: float(X_train[col].median()) for col in X_train.columns}

print(f"    Calculated medians for {len(uncalculable_medians)} uncalculable features")
print(f"    Calculated medians for {len(all_feature_medians)} total features")

# ============================================
# SAVE TO JSON
# ============================================

medians_dict = {
    'uncalculable_features': uncalculable_medians,
    'all_features': all_feature_medians,
    'feature_names': list(X_train.columns),
    'metadata': {
        'n_features_total': len(X_train.columns),
        'n_features_uncalculable': len(uncalculable_features),
        'n_features_calculable': len(X_train.columns) - len(uncalculable_features),
        'training_samples': len(X_train),
        'date_saved': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'best_model': best_final_model_name if 'best_final_model_name' in locals() else 'Unknown'
    }
}

# Save to JSON
medians_file = '../artifacts/feature_engineering/feature_medians.json'

with open(medians_file, 'w') as f:
    json.dump(medians_dict, f, indent=4)

print(f"\n Feature medians saved to: {medians_file}")

# ============================================
# DISPLAY SAMPLE
# ============================================

print(f"\n Sample Uncalculable Feature Medians:")
print(f"{'='*70}")

sample_features = list(uncalculable_medians.items())[:15]
for i, (feature, median) in enumerate(sample_features, 1):
    feature_display = feature[:50] + '...' if len(feature) > 50 else feature
    print(f"   {i:2d}. {feature_display:<53} {median:>10.4f}")

if len(uncalculable_medians) > 15:
    print(f"   ... and {len(uncalculable_medians) - 15} more features")

print(f"\n Feature medians extraction complete!")

In [ ]:
# ============================================
# EXTRACT LOCATION-SPECIFIC FEATURES FOR PRODUCTION
# ============================================

print("\n" + "="*70)
print("EXTRACTING LOCATION-SPECIFIC FEATURES")
print("="*70)

print(f"\n Analyzing locations in training data...")

# ============================================
# 1. CHECK IF WE HAVE LOCATION DATA
# ============================================

# Since we don't have the original df, we'll check if there's a location column in X_train
location_columns = [col for col in X_train.columns if 'place' in col.lower() and 'id' in col.lower()]

if len(location_columns) > 0:
    location_col = location_columns[0]
    print(f" Found location column: {location_col}")
    has_locations = True
else:
    print(" No location column found in X_train.")
    print("   Creating a single 'global' location for all data.")
    has_locations = False
    location_col = 'location'

# ============================================
# 2. IDENTIFY TIME-SERIES FEATURES
# ============================================

# These are features that depend on historical data
time_series_features = [col for col in X_train.columns if any(x in col for x in 
    ['rolling', 'lag', 'diff', 'ewm', 'shift', '_avg', '_std', 
     '_min', '_max', '_median']) and col != location_col]

print(f"\n Time-series features to extract: {len(time_series_features)}")

# ============================================
# 3. EXTRACT FEATURES PER LOCATION
# ============================================

print(f"\n Extracting features for each location...")

location_features = {}

if has_locations:
    # We have actual locations
    unique_locations = X_train[location_col].unique()
    print(f"   Found {len(unique_locations)} unique locations")
    
    for location in unique_locations:
        # Get all rows for this location in training data
        location_mask = X_train[location_col] == location
        location_data = X_train[location_mask]
        
        if len(location_data) == 0:
            continue
        
        # Calculate statistics for time-series features
        location_stats = {}
        
        for feature in time_series_features:
            if feature in location_data.columns:
                # Use the LAST known value (most recent)
                last_value = location_data[feature].iloc[-1]
                
                # Also calculate median (more robust)
                median_value = location_data[feature].median()
                
                # Use last value if available, otherwise median
                if pd.notna(last_value):
                    location_stats[feature] = float(last_value)
                else:
                    location_stats[feature] = float(median_value) if pd.notna(median_value) else 0.0
        
        # Store for this location
        location_features[str(location)] = {
            'features': location_stats,
            'n_samples': int(len(location_data)),
            'last_seen': 'unknown'
        }
    
    print(f" Extracted features for {len(location_features)} locations")
    
else:
    # No location column - create one global location
    print("   Creating single global location...")
    
    location_stats = {}
    
    for feature in time_series_features:
        if feature in X_train.columns:
            # Use last value
            last_value = X_train[feature].iloc[-1]
            
            if pd.notna(last_value):
                location_stats[feature] = float(last_value)
            else:
                median_value = X_train[feature].median()
                location_stats[feature] = float(median_value) if pd.notna(median_value) else 0.0
    
    location_features['global'] = {
        'features': location_stats,
        'n_samples': len(X_train),
        'last_seen': 'unknown'
    }
    
    print(f" Created global location with {len(X_train)} samples")

# ============================================
# 4. CALCULATE GLOBAL FALLBACK (for new locations)
# ============================================

print(f"\n Calculating global fallback features...")

global_features = {}

for feature in time_series_features:
    if feature in X_train.columns:
        # Use median across all data
        median_value = X_train[feature].median()
        global_features[feature] = float(median_value) if pd.notna(median_value) else 0.0

print(f" Calculated {len(global_features)} global fallback features")

# ============================================
# 5. CREATE FINAL LOOKUP STRUCTURE
# ============================================

location_lookup = {
    'locations': location_features,
    'global_fallback': global_features,
    'metadata': {
        'n_locations': len(location_features),
        'n_time_series_features': len(time_series_features),
        'location_column': location_col,
        'has_location_data': has_locations,
        'total_training_samples': len(X_train),
        'date_created': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'feature_list': time_series_features
    }
}

# ============================================
# 6. SAVE TO JSON
# ============================================

lookup_file = '../artifacts/feature_engineering/location_features_lookup.json'

with open(lookup_file, 'w') as f:
    json.dump(location_lookup, f, indent=4)

print(f"\n Location features saved to: {lookup_file}")

# ============================================
# 7. SAVE LOCATION LIST (for dropdown in UI)
# ============================================

location_ids = list(location_features.keys())

location_list = {
    'locations': [
        {
            'id': str(loc),
            'name': str(loc) if loc != 'global' else 'Global (All Data)',
            'n_samples': location_features[str(loc)]['n_samples'],
            'last_seen': location_features[str(loc)]['last_seen']
        }
        for loc in location_ids
    ],
    'metadata': {
        'n_locations': len(location_ids),
        'default_location': str(location_ids[0]) if len(location_ids) > 0 else 'global',
        'has_location_data': has_locations
    }
}

location_list_file = '../artifacts/feature_engineering/available_locations.json'

with open(location_list_file, 'w') as f:
    json.dump(location_list, f, indent=4)

print(f" Available locations saved to: {location_list_file}")

# ============================================
# 8. DISPLAY SUMMARY
# ============================================

print(f"\n" + "="*70)
print(f" LOCATION FEATURES EXTRACTION SUMMARY")
print(f"="*70)

print(f"\n Files Created:")
print(f"   1. {lookup_file}")
print(f"      - Features for {len(location_features)} location(s)")
print(f"      - {len(time_series_features)} time-series features per location")
print(f"   2. {location_list_file}")
print(f"      - List of available locations for UI dropdown")

if has_locations:
    print(f"\n Sample Locations:")
    sample_locations = list(location_features.keys())[:5]
    for i, loc in enumerate(sample_locations, 1):
        n_samples = location_features[loc]['n_samples']
        print(f"   {i}. Location: {loc}")
        print(f"      - Training samples: {n_samples}")
    
    if len(location_features) > 5:
        print(f"   ... and {len(location_features) - 5} more locations")
else:
    print(f"\n Location Data:")
    print(f"    Single 'global' location created")
    print(f"     All training data treated as one location")
    print(f"     In production, all predictions will use same historical features")

print(f"\n Sample Time-Series Features:")
first_loc = list(location_features.keys())[0]
sample_features = list(location_features[first_loc]['features'].items())[:10]

for i, (feature, value) in enumerate(sample_features, 1):
    feature_display = feature[:50] + '...' if len(feature) > 50 else feature
    print(f"   {i:2d}. {feature_display:<53} {value:>10.4f}")

if len(location_features[first_loc]['features']) > 10:
    print(f"   ... and {len(location_features[first_loc]['features']) - 10} more features")

